In [1]:
import torch
print(torch.cuda.is_available())

import os
import zarr
import dask.array as da
from rechunker import rechunk
from numcodecs import Blosc

import napari
import numpy as np
from tqdm import tqdm
from rich.pretty import pprint


from ultrack import track, to_tracks_layer, tracks_to_zarr
from ultrack.utils import labels_to_edges
from ultrack.config import MainConfig


True


In [9]:
# pathway to labels
zarr_masks_path = r'D:\kasia\tracking\E6_exp\E6_small_masks.zarr'

# restrict to a subset of the data for testing (optional)
frames_start = 115
frames_stop = 125
row_start = 4250
row_stop = 4750
col_start = 4200
col_stop = 4900

# frames_start = 0
# frames_stop = -1
# row_start = 0
# row_stop = -1
# col_start = 0
# col_stop = -1

# pathway to save tracked (re-labelled) masks
tracked_path = r'D:\kasia\tracking\E6_exp\labels_test_241001.zarr'
# chunk size for the tracked masks
target_chunks = (1, 300, 300)  

In [3]:
# tracking parameters

config = MainConfig()
config.segmentation_config.min_area = 400
config.segmentation_config.max_area = 2000
config.segmentation_config.min_frontier = 0.1
config.segmentation_config.n_workers = 10

config.linking_config.max_distance = 25
config.linking_config.n_workers = 10

config.tracking_config.appear_weight = -1
config.tracking_config.disappear_weight = -1
config.tracking_config.division_weight = -0.1
config.tracking_config.power = 4
config.tracking_config.bias = -0.001
config.tracking_config.solution_gap = 0.0

config.tracking_config.window_size = 50
config.tracking_config.overlap_size = 5

pprint(config)

MainConfig(
│   data_config=DataConfig(working_dir=WindowsPath('.'), database='sqlite', address=None, n_workers=1),
│   segmentation_config=SegmentationConfig(
│   │   threshold=0.5,
│   │   min_area=400,
│   │   max_area=2000,
│   │   min_frontier=0.1,
│   │   anisotropy_penalization=0.0,
│   │   max_noise=0.0,
│   │   ws_hierarchy=<function watershed_hierarchy_by_area at 0x000001A1FFB264D0>,
│   │   n_workers=10
│   ),
│   linking_config=LinkingConfig(
│   │   n_workers=10,
│   │   max_neighbors=5,
│   │   max_distance=25,
│   │   distance_weight=0.0,
│   │   z_score_threshold=5.0
│   ),
│   tracking_config=TrackingConfig(
│   │   appear_weight=-1,
│   │   disappear_weight=-1,
│   │   division_weight=-0.1,
│   │   dismiss_weight_guess=None,
│   │   include_weight_guess=None,
│   │   window_size=50,
│   │   overlap_size=5,
│   │   solution_gap=0.0,
│   │   time_limit=36000,
│   │   method=0,
│   │   n_threads=-1,
│   │   link_function='power',
│   │   power=4,
│   │   bias=-0.001
│   )
)

In [4]:
# change the working directory
os.chdir(os.path.dirname(zarr_masks_path))

# read in masks
dask_masks = da.from_zarr(zarr_masks_path,0)[frames_start:frames_stop, row_start:row_stop, col_start:col_stop]
dask_masks.shape

(10, 500, 700)

In [5]:
# convert labels to detections and edges
# they will be automatically saved to the working directory

detections, edges = labels_to_edges(dask_masks, 
                                   detection_store_or_path="detection_test.zarr", 
                                   edges_store_or_path="edges_test.zarr", 
                                   overwrite=True, 
                                   sigma=4.0)

c:\Users\kmk280\AppData\Local\miniconda3\envs\tracking\lib\site-packages\ultrack\utils\array.py:185: FutureWarning: The NestedDirectoryStore is deprecated and will be removed in a Zarr-Python version 3, see https://github.com/zarr-developers/zarr-python/issues/1274 for more information.
  store = zarr.NestedDirectoryStore(str(store_or_path))
Converting labels to edges: 100%|██████████| 10/10 [00:00<00:00, 19.76it/s]


In [6]:
# tracking from detections and edges

track(
    detection=detections,
    edges=edges,
    config=config,
    overwrite=True, #"solutions" - to recalculate from the database
)

Linking nodes.: 100%|██████████| 9/9 [00:09<00:00,  1.09s/it]


Using Gurobi solver
Solving ILP batch 0
Constructing ILP ...
Solving ILP ...
Saving solution ...
Done!


In [7]:
# extract labels

tracks_df, graph = to_tracks_layer(config)
labels = tracks_to_zarr(config, tracks_df)

c:\Users\kmk280\AppData\Local\miniconda3\envs\tracking\lib\site-packages\numba\typed\typeddict.py:39: NumbaTypeSafetyWarning: unsafe cast from int64 to int32. Precision may be lost.
  return d[key]
Exporting segmentation masks: 100%|██████████| 10/10 [00:00<00:00, 136.99it/s]


In [24]:
# Define the target store and array parameters
target_store = zarr.DirectoryStore(tracked_path, dimension_separator="/") 


compressor = Blosc(cname='lz4', clevel=5, shuffle=Blosc.SHUFFLE, blocksize=0)

rechunked = rechunk(labels, 
                    max_mem=500000000,
                    target_chunks=target_chunks, 
                    target_store=target_store,
                    target_options={'compressor': compressor},
                    )

rechunked.execute()

<zarr.core.Array (10, 500, 700) int32>

In [53]:
# track(
#     labels= [dask_masks, dask_masks2],
#     config=config,
#     overwrite=True,
# )

Linking nodes.: 100%|██████████| 9/9 [00:10<00:00,  1.13s/it]


Using Gurobi solver
Solving ILP batch 0
Constructing ILP ...
Solving ILP ...
Saving solution ...
Done!


In [12]:
import zarr

import dask.array as da

In [23]:
labels

<zarr.core.Array (10, 500, 700) int32>

In [25]:

labels = da.from_zarr(target_path)
labels.shape

(10, 500, 700)

In [26]:
labels

dask.array<from-zarr, shape=(10, 500, 700), dtype=int32, chunksize=(1, 300, 300), chunktype=numpy.ndarray>

In [13]:
target_path = r'D:\kasia\tracking\E6_exp\double_segmentation_ultrack\labels_double_flat.zarr'
labels_path = r'D:\kasia\tracking\E6_exp\double_segmentation_ultrack\temp_store.zarr'
labels = zarr.open(labels_path, mode='r')

target_store = zarr.DirectoryStore(target_path, dimension_separator=".") 
target_chunks = (1, labels.shape[1], labels.shape[2])  # New chunk shape

compressor = Blosc(cname='lz4', clevel=5, shuffle=Blosc.SHUFFLE, blocksize=0)

rechunked = rechunk(labels, 
                    max_mem=500000000,
                    target_chunks=target_chunks, 
                    target_store=target_store,
                    target_options={'compressor': compressor},
                    )

rechunked.execute()

ContainsArrayError: path '' contains an array

In [14]:
labels.info

Type,zarr.core.Array
Data type,int32
Shape,"(240, 8395, 8400)"
Chunk shape,"(1, 8395, 8400)"
Order,C
Read-only,False
Compressor,"Blosc(cname='lz4', clevel=5, shuffle=SHUFFLE, blocksize=0)"
Store type,zarr.storage.TempStore
No. bytes,67697280000 (63.0G)
No. bytes stored,1759328863 (1.6G)
Storage ratio,38.5


In [13]:
from numcodecs import Blosc
import zarr
from ome_zarr.io import parse_url
from ome_zarr.writer import write_image

compressor = Blosc(cname='lz4', clevel=1, shuffle=Blosc.BITSHUFFLE)

# save zarr file

size_t = 10
size_xy = 128

zarr_path = r'D:\kasia\tracking\zarr_tests\labels_double.zarr'

# write the image data
store = parse_url(zarr_path, mode="w").store
root = zarr.group(store=store)

# it will fail if the store already contains arrays
write_image(image=labels, group=root, axes="tyx", storage_options=dict(chunks=(size_t,size_xy, size_xy),compressor=compressor))

KeyboardInterrupt: 

In [ ]:
from numcodecs import Blosc
import zarr
from ome_zarr.io import parse_url
from ome_zarr.writer import write_image

compressor = Blosc(cname='lz4', clevel=9, shuffle=Blosc.BITSHUFFLE)

# save zarr file

size_t = 10
size_xy = 128

zarr_path = r'D:\kasia\tracking\zarr_tests\labels_double_c9.zarr'

# write the image data
store = parse_url(zarr_path, mode="w").store
root = zarr.group(store=store)

# it will fail if the store already contains arrays
write_image(image=labels, group=root, axes="tyx", storage_options=dict(chunks=(size_t,size_xy, size_xy),compressor=compressor))

: 

In [ ]:
from numcodecs import Blosc
import zarr
from ome_zarr.io import parse_url
from ome_zarr.writer import write_image

compressor = Blosc(cname='lz4', clevel=9, shuffle=Blosc.BITSHUFFLE)

# save zarr file

size_t = 1
size_xy = 512

zarr_path = r'D:\kasia\tracking\zarr_tests\labels_double_c9_512.zarr'

# write the image data
store = parse_url(zarr_path, mode="w").store
root = zarr.group(store=store)

# it will fail if the store already contains arrays
write_image(image=labels, group=root, axes="tyx", storage_options=dict(chunks=(size_t,size_xy, size_xy),compressor=compressor))

In [55]:
viewer = napari.Viewer()
viewer.add_image(ch0_da, name="ch0", colormap="green", blending="additive")
viewer.add_image(ch1_da, name="ch1", colormap="red", blending="additive")
viewer.add_labels(dask_masks)
viewer.add_image(edges, blending="additive", colormap="magma")
viewer.add_image(edges2, blending="additive", colormap="magma")
viewer.add_labels(labels)


c:\Users\kmk280\.conda\envs\ultrack-env\lib\site-packages\napari\_qt\qt_main_window.py:629: UserWarning: Using NAPARI_PERFMON with an already-running QtApp (--gui qt?) is not supported.
  qapp = get_app()


<Labels layer 'labels' at 0x1ed110d9660>

In [52]:
viewer.add_labels(dask_masks2)

<Labels layer 'dask_masks2' at 0x1ed2ab49ab0>